# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 20 2022 3:19PM,242366,10,MSP212408,"Methapharm, Inc.",Released
1,Jul 20 2022 3:19PM,242366,10,MSP212411,"Methapharm, Inc.",Released
2,Jul 20 2022 3:19PM,242366,10,MSP212412,"Methapharm, Inc.",Released
3,Jul 20 2022 3:19PM,242366,10,MSP212413,"Methapharm, Inc.",Released
4,Jul 20 2022 3:19PM,242366,10,MSP212414,"Methapharm, Inc.",Released
5,Jul 20 2022 3:19PM,242366,10,MSP212415,"Methapharm, Inc.",Released
6,Jul 20 2022 3:19PM,242366,10,MSP212416,"Methapharm, Inc.",Released
7,Jul 20 2022 3:19PM,242366,10,MSP212417,"Methapharm, Inc.",Released
8,Jul 20 2022 3:19PM,242366,10,MSP212418,"Methapharm, Inc.",Released
9,Jul 20 2022 3:19PM,242366,10,MSP212409,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,242361,Released,1
39,242362,Released,1
40,242363,Released,11
41,242364,Released,2
42,242366,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242361,NaN,NaN,1.0
242362,NaN,NaN,1.0
242363,NaN,NaN,11.0
242364,NaN,NaN,2.0
242366,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242263,0.0,0.0,12.0
242281,3.0,24.0,5.0
242282,0.0,0.0,1.0
242290,0.0,0.0,1.0
242291,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242263,0,0,12
242281,3,24,5
242282,0,0,1
242290,0,0,1
242291,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242263,0,0,12
1,242281,3,24,5
2,242282,0,0,1
3,242290,0,0,1
4,242291,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242263,,,12
1,242281,3,24,5
2,242282,,,1
3,242290,,,1
4,242291,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 20 2022 3:19PM,242366,10,"Methapharm, Inc."
12,Jul 20 2022 3:18PM,242364,10,"Methapharm, Inc."
14,Jul 20 2022 3:16PM,242363,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
25,Jul 20 2022 3:08PM,242362,19,Eli Lilly and Company
26,Jul 20 2022 2:57PM,242361,12,Doctors Scientific Organica
27,Jul 20 2022 2:32PM,242359,10,Bio-PRF
35,Jul 20 2022 2:31PM,242358,10,Emerginnova
40,Jul 20 2022 2:20PM,242357,10,Eywa Pharma Inc.
48,Jul 20 2022 2:19PM,242341,10,Eywa Pharma Inc.
52,Jul 20 2022 2:08PM,242354,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 20 2022 3:19PM,242366,10,"Methapharm, Inc.",,,12
1,Jul 20 2022 3:18PM,242364,10,"Methapharm, Inc.",,,2
2,Jul 20 2022 3:16PM,242363,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,11
3,Jul 20 2022 3:08PM,242362,19,Eli Lilly and Company,,,1
4,Jul 20 2022 2:57PM,242361,12,Doctors Scientific Organica,,,1
5,Jul 20 2022 2:32PM,242359,10,Bio-PRF,,,8
6,Jul 20 2022 2:31PM,242358,10,Emerginnova,,1,4
7,Jul 20 2022 2:20PM,242357,10,Eywa Pharma Inc.,,,8
8,Jul 20 2022 2:19PM,242341,10,Eywa Pharma Inc.,,,4
9,Jul 20 2022 2:08PM,242354,19,ACG North America LLC,,1,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 3:19PM,242366,10,"Methapharm, Inc.",12,,
1,Jul 20 2022 3:18PM,242364,10,"Methapharm, Inc.",2,,
2,Jul 20 2022 3:16PM,242363,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11,,
3,Jul 20 2022 3:08PM,242362,19,Eli Lilly and Company,1,,
4,Jul 20 2022 2:57PM,242361,12,Doctors Scientific Organica,1,,
5,Jul 20 2022 2:32PM,242359,10,Bio-PRF,8,,
6,Jul 20 2022 2:31PM,242358,10,Emerginnova,4,1,
7,Jul 20 2022 2:20PM,242357,10,Eywa Pharma Inc.,8,,
8,Jul 20 2022 2:19PM,242341,10,Eywa Pharma Inc.,4,,
9,Jul 20 2022 2:08PM,242354,19,ACG North America LLC,2,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 3:19PM,242366,10,"Methapharm, Inc.",12,,
1,Jul 20 2022 3:18PM,242364,10,"Methapharm, Inc.",2,,
2,Jul 20 2022 3:16PM,242363,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11,,
3,Jul 20 2022 3:08PM,242362,19,Eli Lilly and Company,1,,
4,Jul 20 2022 2:57PM,242361,12,Doctors Scientific Organica,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 3:19PM,242366,10,"Methapharm, Inc.",12.0,NaN,NaN
1,Jul 20 2022 3:18PM,242364,10,"Methapharm, Inc.",2.0,NaN,NaN
2,Jul 20 2022 3:16PM,242363,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11.0,NaN,NaN
3,Jul 20 2022 3:08PM,242362,19,Eli Lilly and Company,1.0,NaN,NaN
4,Jul 20 2022 2:57PM,242361,12,Doctors Scientific Organica,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 20 2022 3:19PM,242366,10,"Methapharm, Inc.",12.0,0.0,0.0
1,Jul 20 2022 3:18PM,242364,10,"Methapharm, Inc.",2.0,0.0,0.0
2,Jul 20 2022 3:16PM,242363,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,11.0,0.0,0.0
3,Jul 20 2022 3:08PM,242362,19,Eli Lilly and Company,1.0,0.0,0.0
4,Jul 20 2022 2:57PM,242361,12,Doctors Scientific Organica,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3634938,80.0,30.0,3.0
12,484697,2.0,0.0,0.0
16,1211514,4.0,3.0,0.0
19,2181004,46.0,21.0,0.0
20,242349,4.0,0.0,0.0
21,969299,4.0,0.0,0.0
22,242329,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3634938,80.0,30.0,3.0
1,12,484697,2.0,0.0,0.0
2,16,1211514,4.0,3.0,0.0
3,19,2181004,46.0,21.0,0.0
4,20,242349,4.0,0.0,0.0
5,21,969299,4.0,0.0,0.0
6,22,242329,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,80.0,30.0,3.0
1,12,2.0,0.0,0.0
2,16,4.0,3.0,0.0
3,19,46.0,21.0,0.0
4,20,4.0,0.0,0.0
5,21,4.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,80.0
1,12,Released,2.0
2,16,Released,4.0
3,19,Released,46.0
4,20,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,30.0,0.0,3.0,21.0,0.0,0.0,0.0
Released,80.0,2.0,4.0,46.0,4.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,30.0,0.0,3.0,21.0,0.0,0.0,0.0
2,Released,80.0,2.0,4.0,46.0,4.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,30.0,0.0,3.0,21.0,0.0,0.0,0.0
2,Released,80.0,2.0,4.0,46.0,4.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()